# Model Test Notebook

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path
from num2words import num2words
import re

## Loading the Data

In [8]:
root_dirname = os.path.abspath('')
#ted_talk_scraper_dir = os.path.join(root_dirname, 'Ted-Talk_Scraper')
audio_folders = os.path.join(root_dirname, 'output_files')

train_audio_path = os.path.join(audio_folders, '2006-sir-ken-robinson-016-1200k')
train_label_path = os.path.join(root_dirname, 'transcript', '2006-sir-ken-robinson-016-1200k.csv')

print("root_dirname:", root_dirname)
print("ted_talk_scraper_dir:", ted_talk_scraper_dir)
print("audio_folders:", audio_folders)
print("train_audio_path:", train_audio_path)
print("train_label_path:", train_label_path)

root_dirname: C:\Users\dangn\Documents\Senior Year\CS 4980 NLP\Ted-Talk_Scraper
ted_talk_scraper_dir: C:\Users\dangn\Documents\Senior Year\CS 4980 NLP\Ted-Talk_Scraper\Ted-Talk_Scraper
audio_folders: C:\Users\dangn\Documents\Senior Year\CS 4980 NLP\Ted-Talk_Scraper\output_files
train_audio_path: C:\Users\dangn\Documents\Senior Year\CS 4980 NLP\Ted-Talk_Scraper\output_files\2006-sir-ken-robinson-016-1200k
train_label_path: C:\Users\dangn\Documents\Senior Year\CS 4980 NLP\Ted-Talk_Scraper\transcript\2006-sir-ken-robinson-016-1200k.csv


## Cleaning the Data

So far the data is cleaned by the following:
- Converted numbers to words using num2word module (e.g. 30s -> thirtys). Not fully correct but decent
- Removed punctuation
- lowercased all
- removed row entries in train_df where transcript was just audience sound (e.g. (Applause))

In [7]:
def convert_num_to_words(utterance):
    numbers = re.findall("\d+", utterance)
    for num in sorted([int(i) for i in numbers], reverse=True):
        utterance  = utterance.replace(str(num), num2words(num))
    return utterance

transcript_df = pd.read_csv(train_label_path)
audience_sound_only_mask = transcript_df['text'].str.contains('^\s?\(\w+\)\s?$', regex=True)
transcript_df['text'] = transcript_df['text'].apply(convert_num_to_words)
transcript_df = transcript_df[~audience_sound_only_mask]['text'].str.lower().str.replace('\(\w+\)', '', regex=True).str.replace("[\"\?\.\!\:\,\-\;\-\']", "", regex=True)

audio_file_name = os.listdir(train_audio_path)
audio_file_size = []
for audio in audio_file_name:
    audio_file_size.append(Path(os.path.join(train_audio_path,  audio)).stat().st_size)
    
train_df = pd.DataFrame({"wav_filename":audio_file_name, "wav_filesize":audio_file_size})
train_df = pd.merge(train_df, transcript_df.rename('transcript'), left_index=True, right_index=True)
display(train_df)
train_df.to_csv("train.csv", index=False)

,wav_filename,wav_filesize,transcript
0,output_0.mp3,32852,good morning how are you
1,output_1.mp3,32852,good
2,output_10.mp3,48735,its been great hasnt it ive been blown away by...
4,output_101.mp3,320826,there have been three themes running through t...
5,output_102.mp3,496787,i have an interest in education actually what ...
...,...,...,...
98,output_94.mp3,32852,i said what happened she said she did i cant t...
100,output_96.mp3,544852,what i think it comes to is this al gore spoke...
101,output_97.mp3,368892,there was a wonderful quote by jonas salk who ...
102,output_98.mp3,656865,what ted celebrates is the gift of the human i...
